In [69]:
pip install pyspark


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
#initiate spark instance 
import pyspark
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
#Create SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkApp") \
    .getOrCreate()

#spark.stop()
print("spark success")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/21 22:40:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark success


24/07/21 22:40:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
#print current directory
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current Working Directory: {current_directory}")


Current Working Directory: /Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/SparkA1


In [4]:
#part 1:1.load all csv in this same path
df= spark.read.format("csv").option("header", True).option("inferSchema", True).load("/Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/lab_dataset.csv")
#df.show()
#display(df.schema) 


In [71]:
#part1:2 Allow the user to search the dataset by the borough name, showing the total number of callouts in 2020.
# Filter the DataFrame to include only data for the year 2020
from pyspark.sql.functions import lit,upper, levenshtein


df_2020 = df.filter(col('CalYear') == 2020)
# Group the data by 'IncGeo_BoroughName' and sum the 'NumbCalls' column
df_Borough_call = df_2020.groupBy('IncGeo_BoroughName', 'ProperCase').sum('NumCalls')\
                         .selectExpr( "`ProperCase` as `Borough Name CaseCorrect` ", "`sum(NumCalls)` as `Total_number_of_callouts`",)

#df_Borough_call.show(n=33, truncate=False)
# Function to search by borough name
def search_borough():
    userinput = input("Enter the borough name: ").strip()
    borough_input = userinput.upper()
    #give error if input is not letters
    if not borough_input.isalpha():
        print ("Borough name should be a string.")
        return
    try:
        result = df_Borough_call.filter(upper(col('Borough Name CaseCorrect')) == borough_input)
        # give error if the borough name is not in the dataset
        if result.count() == 0:
            # Check for the borough name is partial match
            partialmatch =df_Borough_call.filter(upper(col('Borough Name CaseCorrect')).contains(borough_input))
            if partialmatch.count() > 0:
                # Collect the matching borough names into a single list
                match_names = [row['Borough Name CaseCorrect'] for row in partialmatch.select('Borough Name CaseCorrect').collect()]
                # Print the matches in one line
                print(f"Partial match found. Did you mean: {', '.join(match_names)}")
                partialmatch.show(truncate=False)
            else:
                # Calculate Levenshtein distance for each borough name
                distances = df_Borough_call.withColumn('distance', levenshtein(upper(col('Borough Name CaseCorrect')), lit(borough_input)))
                # Find the closest match based on minimum distance
                closest_match = distances.select('Borough Name CaseCorrect', 'distance').orderBy('distance').first()
                closest_borough_name = closest_match['Borough Name CaseCorrect']
                print(f"No exact match found for '{userinput}'. Did you mean '{closest_borough_name}'? ")
                df_Borough_call.filter(col('Borough Name CaseCorrect') == closest_borough_name).show(truncate=False)
            return
        else:
            result.show(truncate=False)
    except Exception as e:
        print(f"Error: {e}")
# Call the function to take user input and display the result
search_borough()

Partial match found. Did you mean: City Of london
+------------------------+------------------------+
|Borough Name CaseCorrect|Total_number_of_callouts|
+------------------------+------------------------+
|City Of london          |1040                    |
+------------------------+------------------------+



In [73]:
# Wrong match demo for calculating distance

df_2020 = df.filter(col('CalYear') == 2020)
# Group the data by 'IncGeo_BoroughName' and sum the 'NumbCalls' column
df_Borough_call = df_2020.groupBy('IncGeo_BoroughName').sum('NumCalls')
df_Borough_call = df_Borough_call.withColumnRenamed('IncGeo_BoroughName', 'Borough Name')\
                                 .withColumnRenamed('sum(NumCalls)', 'Total number of callouts')

#df_Borough_call.show(n=33, truncate=False)
# Function to search by borough name
def search_borough():
    borough_name = input("Enter the borough name: ").strip().upper()
    #give error if input is not letters
    if not borough_name.isalpha():
        print ("Borough name should be a string.")
        return
    try:
        result = df_Borough_call.filter(col('Borough Name') == borough_name)
        # give error if the borough name is not in the dataset
        if result.count() == 0:
            # Calculate Levenshtein distance for each borough name
            distances = df_Borough_call.withColumn('distance', levenshtein(upper(col('Borough Name')), lit(borough_name)))
            #distances.show(truncate=False)
            # Find the closest match based on minimum distance
            closest_match = distances.select('Borough Name', 'distance').orderBy('distance').first()
            closest_borough_name = closest_match['Borough Name']

            
            print(f"No exact match found for '{borough_name}'. Did you mean '{closest_borough_name}'? ")
       
        else:
            result.show(truncate=False)
    
    except Exception as e:
        print(f"Error: {e}")
# Call the function to take user input and display the result
search_borough()

No exact match found for 'LONDON'. Did you mean 'CROYDON'? 


In [7]:
#part1: 3  Given a range of hours (for example between 1-2 hours), show the user all incidents where the LFB spent this much time dealing with it.
# Define the hours of interest as a list
hours_of_interest = [48,49]

# Filter DataFrame based on the HourOfCall and IncidentGroup conditions
df_time_spend = df.filter(
    (col("PumpHoursRoundUp").isin(hours_of_interest)) 
)
print(f"There are in total {df_time_spend.count()} incidents happened in 2021 which London Fire Brigade spent {hours_of_interest} hours dealing with it. Here are the detials:")
df_time_spend.show(truncate= False)

There are in total 15 incidents happened in 2021 which London Fire Brigade spent [48, 49] hours dealing with it. Here are the detials:


+---------------+-----------+-------+-------------------+----------+---------------+-------------------+---------------------------+----------------+-----------------------------------------------------+--------------------------------------------+-------------+-----------------+---------------+--------+------------------+--------------------+--------------------+---------------+------------------------------+------------------------------+---------+----------+---------------+----------------+-----------+------------+------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+----------------+-----------------+--------+
|IncidentNumber |DateOfCall |CalYear|TimeOfCall         |HourOfCall|IncidentGroup  |StopCodeDescription|SpecialServiceType         |PropertyCategory|PropertyType                                        

In [8]:
#part1: 4 Calculate the number of incidents in 2021 where there were at least 2 fire pumps deployed.
# Filter DataFrame where PumpCount > 2
df_condition = df.filter((col("PumpCount") >=2 ) & (col("CalYear") == 2021)) 

# Count distinct PumpCount values
IncidentNumber = df_condition.select("IncidentNumber").distinct().count()
print(f"There are number of {IncidentNumber} incidents happened in 2021 which London Fire Brigade deployed at least 2 fire pumps")


There are number of 41572 incidents happened in 2021 which London Fire Brigade deployed at least 2 fire pumps


In [8]:
#part1:5.Which type of property had the most incident callouts during 2022?


from pyspark.sql.functions import col
from pyspark.sql import functions as F
# Filter the DataFrame to include only data for the year 2020
df_2022 = df.filter(col('CalYear') == 2022)
# Getting  the total number of incident calls for each property type
df_Property = df_2022.groupBy('PropertyType').agg(F.sum('NumCalls').alias('TotalNumCalls'))
df_Property = df_Property.orderBy(F.col('TotalNumCalls').desc())

df_Property_first =df_Property.first()
print(f"Property type [ {df_Property_first['PropertyType']}] had the most incident callouts during 2022")
 


Property type [ Purpose Built Flats/Maisonettes - 4 to 9 storeys ] had the most incident callouts during 2022


In [5]:
# Using an appropriate Python library, 
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022. 
df_plot = df.filter(col("CalYear").isin([2020,2021,2022]))
# Make sure to define how you interpret a road incident based on the dataset.

#define road incident: road incident use PropertyCatergory as Road Vehicle, but there is also Road incident that happens without a vechile involved. 
#Therefoore PropertyType = Road surface/pavement, Roadside vegetation, Roadside furniture, Bridge. and PropertyCatergory = Road Vehicle
road_incidents = df_plot.filter(
    (col("PropertyCategory") == "Road Vehicle") | 
    (col("PropertyType").isin("Road surface/pavement", "Roadside vegetation", "Roadside furniture", "Bridge")) # these are in PropertyCatergory = "Outdoor"
)

#20629 with Road Vehicle Catergory, and with other condition, it is in 20988

# draw a graph showing the trend of incident callouts on the road between 2020 and 2022
import plotly.express as px
fig = px.line(road_incidents, x="DateOfCall", y="NumCalls", title="Trend of incident callouts on the road between 2020 and 2022")
# Update x-axis to show month marks only
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Calls",
    xaxis=dict(
        tickformat="%b ",  # Format the ticks to show month and year
        dtick="M1"           # Show ticks every month
    )
)
fig.show()

24/07/21 22:45:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [41]:
# Using an appropriate Python library, 
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022. 
df_plot = df.filter(col("CalYear").isin([2020,2021,2022]))
# Make sure to define how you interpret a road incident based on the dataset.

#define road incident: road incident use PropertyCatergory as Road Vehicle, but there is also Road incident that happens without a vechile involved. 
#Therefoore PropertyType = Road surface/pavement, Roadside vegetation, Roadside furniture, Bridge. and PropertyCatergory = Road Vehicle
road_incidents = df_plot.filter(
    (col("PropertyCategory") == "Road Vehicle") | 
    (col("PropertyType").isin("Road surface/pavement", "Roadside vegetation", "Roadside furniture", "Bridge")) # these are in PropertyCatergory = "Outdoor"
)
from pyspark.sql.functions import  to_date, count,month, concat, lit
#20629 with Road Vehicle Catergory, and with other condition, it is in 20988

# Convert the 'DateOfCall' column from string to DateType
road_incidents = road_incidents.withColumn("DateOfCall", to_date(road_incidents["DateOfCall"], "dd MMM yyyy"))

road_incidents = road_incidents.withColumn("MonthOfCall",  concat(month(road_incidents["DateOfCall"]).cast("integer")))
road_incidents = road_incidents.withColumn("YearMonthOfCall",concat(col("CalYear"), lit("-"), col("MonthOfCall")))

# # Group by year and month and count the number of records for each month
df_monthly_counts = road_incidents.groupBy("CalYear","MonthOfCall","YearMonthOfCall").agg(count("NumCalls").alias("NumCallsMonthly"))
df_monthly_counts = df_monthly_counts.withColumn("MonthOfCall", col("MonthOfCall").cast("integer"))
df_monthly_counts = df_monthly_counts.orderBy("CalYear","MonthOfCall")
df_monthly_counts.show()

import plotly.express as px

fig = px.line(df_monthly_counts, x="YearMonthOfCall", y="NumCallsMonthly", title="Trend of LFR road incident callouts on the road between 2020 and 2022")


fig.show()


+-------+-----------+---------------+---------------+
|CalYear|MonthOfCall|YearMonthOfCall|NumCallsMonthly|
+-------+-----------+---------------+---------------+
|   2020|          1|         2020-1|            616|
|   2020|          2|         2020-2|            543|
|   2020|          3|         2020-3|            515|
|   2020|          4|         2020-4|            336|
|   2020|          5|         2020-5|            455|
|   2020|          6|         2020-6|            501|
|   2020|          7|         2020-7|            589|
|   2020|          8|         2020-8|            626|
|   2020|          9|         2020-9|            578|
|   2020|         10|        2020-10|            560|
|   2020|         11|        2020-11|            483|
|   2020|         12|        2020-12|            456|
|   2021|          1|         2021-1|            432|
|   2021|          2|         2021-2|            376|
|   2021|          3|         2021-3|            450|
|   2021|          4|       